In [1]:
import numpy as np

########################################################################
#                                                                      #
########################################################################
def convert_lon(lons,reverse=0):
    if reverse == 0:
        return np.fmod((lons + 360), 360.0)
    else:
        return np.fmod((lons + 180), 360.0) - 180

########################################################################
#                                                                      #
########################################################################
def create_enso_timeseries(tskin, lats, lons, lsm):

    # 1. create 2D lons and lats
    lons2d, lats2d = np.meshgrid(lons,lats)

    # calculate the weights
    wgts = np.cos(np.radians(lats2d))
    
    ####################################################################
    # NOW WE CALCULATE THE SST TIME SERIES FOR DIFFERENT ENSO REGIONS
    #
    # 2. define ENSO regions
    # nino regions [min_x, max_x, min_y, max_y]
    # for longitude grid that runs from 0-360
    regions={'Nino_1+2':[convert_lon(-90),
                         convert_lon(-80),
                         -10,0],
             'Nino_3':[convert_lon(-150),
                       convert_lon(-90),
                       -5, 5],
             'Nino_3.4':[convert_lon(-170),
                        convert_lon(-120),
                        -5, 5],
             'Nino_4':[convert_lon(160),
                   convert_lon(-150),
                   -5, 5]}
    # 3. calculate gridded tskin anomolies
    tdim, ydim, xdim = tskin.shape
    enso_ts = {}
   
    
    # loop over each time step and andcalculated weighted mean 
    for tt, grid in enumerate(tskin):
        mdx = tt % 12
        for reg in regions:
            if tt == 0:
                enso_ts[reg] = np.full(tdim, np.nan)
            else:
                pass
            bbox = regions[reg]
            find = np.where((lons2d >= bbox[0])&\
                            (lons2d <= bbox[1])&\
                            (lats2d >= bbox[2])&\
                            (lats2d <= bbox[3])&\
                            (lsm == 0))
            
            wgts = np.cos(np.radians(lats2d[find]))
            enso_ts[reg][tt] = np.nansum(grid[find]*wgts)/np.nansum(wgts)
            


    # calculate weighted mean sst anomoly time series
    sst_anomolies = {}
    for reg in regions:
        sst_anomolies[reg] = np.full(tdim, np.nan)
        for mth in range(12):
            sst_anomolies[reg][mth::12] = enso_ts[reg][mth::12] - np.nanmean(enso_ts[reg][mth::12])
            
    return sst_anomolies